In [ ]:
import fxcmpy
import time
import datetime as dt
from pyti.simple_moving_average import simple_moving_average as sma
from pyti.exponential_moving_average import exponential_moving_average as ema
import talib as ta
import numpy as np

### STRATEGY DESCRIPTION ####
# This strategy buys when the Fast SMA crosses over the Slow SMA and sell when the Fast SMA crosses
# under the Slow SMA. Opposing signals will close out opposing positions if close_on_opposing_signal == True.
# If close_on_opposing_signal == False, the strategy can open hedged positions (long & short at the same time)
# Parameters allow traders to change token, symbol, timeframe, Fast SMA & Slow SMA Periods, 
# Opposing signal closing logic, trade size/stop/limit.
# This is a close-of-bar strategy, meaning it only signals trades at the close of a bar.
# For more strategy examples, please visit github.com/fxcm/RestAPI
#############################

###### USER PARAMETERS ######
# token = '7363fec3b64647198252257812994375c9769f4d'
symbol = 'EUR/USD'
timeframe = "m1"	        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
fast_sma_periods = 10
slow_sma_periods = 20
close_on_opposing_signal = True
amount = 1
stop = -10
trailstop_value = 10
limit = 30
#############################

# Global Variables
pricedata = None
numberofcandles = 300

# Connect to FXCM API
con = fxcmpy.fxcmpy(access_token="33f1a78a9a59a52df59dc8b61b2d6ba7f3b0e7d8", log_level="error")
# con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
	
# This function runs once at the beginning of the strategy to run initial one-time processes/computations
def Prepare():
	global pricedata
	
	print("Requesting Initial Price Data...")
	pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	print(pricedata)
	print("Initial Price Data Received...")

# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
	while True:
		currenttime = dt.datetime.now()
		if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
			Update()
		elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(240)
		elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(840)
		elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
			Update()
			time.sleep(1740)
		elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
			Update()
			time.sleep(3540)
		time.sleep(1)

# Returns True when pricedata is properly updated			
def GetLatestPriceData():
	global pricedata
	
	# Normal operation will update pricedata on first attempt
	new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata= new_pricedata
		return True
		
	counter = 0
	# If data is not available on first attempt, try up to 3 times to update pricedata
	while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
		print("No updated prices found, trying again in 10 seconds...")
		counter+=1
		time.sleep(10)
		new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata = new_pricedata
		return True
	else:
		return False
		
# This function is run every time a candle closes
def Update():
	print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")
    
	colfastL = False
	colfastS = False
    
	colslowL = False
	colslowS = False
    
	colFinal = ""
	colFinal2 = ""
    
	closes = np.array(pricedata['bidclose'])
    
    #Fast EMA ARRAYS
	ema1array = ta.EMA(closes, timeperiod=3)
	ema2array = ta.EMA(closes, timeperiod=6)  
	ema3array = ta.EMA(closes, timeperiod=9)  
	ema4array = ta.EMA(closes, timeperiod=12) 
	ema5array = ta.EMA(closes, timeperiod=15) 
	ema6array = ta.EMA(closes, timeperiod=18) 
	ema7array = ta.EMA(closes, timeperiod=21)
    
    #Slow EMA ARRAYS
	ema8array = ta.EMA(closes, timeperiod=24)
	ema9array = ta.EMA(closes, timeperiod=27)
	ema10array = ta.EMA(closes, timeperiod=30)
	ema11array = ta.EMA(closes, timeperiod=33)
	ema12array = ta.EMA(closes, timeperiod=36)
	ema13array = ta.EMA(closes, timeperiod=39)
	ema14array = ta.EMA(closes, timeperiod=42)
	ema15array = ta.EMA(closes, timeperiod=45)
	ema16array = ta.EMA(closes, timeperiod=48)
	ema17array = ta.EMA(closes, timeperiod=51)
	ema18array = ta.EMA(closes, timeperiod=54)
	ema19array = ta.EMA(closes, timeperiod=57)
	ema20array = ta.EMA(closes, timeperiod=60)
	ema21array = ta.EMA(closes, timeperiod=63)
	ema22array = ta.EMA(closes, timeperiod=66)
    
    #EMA 200 ARRAY
	ema23array = ta.EMA(closes, timeperiod=200)
    
    
    #Fast EMA
	ema1 = ema1array[len(ema1array)-1]
	ema2 = ema2array[len(ema2array)-1]
	ema3 = ema3array[len(ema3array)-1]
	ema4 = ema4array[len(ema4array)-1]
	ema5 = ema5array[len(ema5array)-1]
	ema6 = ema6array[len(ema6array)-1]
	ema7 = ema7array[len(ema7array)-1]
    
    
    #Slow EMA
	ema8 = ema8array[len(ema8array)-1]
	ema9 = ema9array[len(ema9array)-1]
	ema10 = ema10array[len(ema10array)-1]
	ema11 = ema11array[len(ema11array)-1]
	ema12 = ema12array[len(ema12array)-1]
	ema13 = ema13array[len(ema13array)-1]
	ema14 = ema14array[len(ema14array)-1]
	ema15 = ema15array[len(ema15array)-1]
	ema16 = ema16array[len(ema16array)-1]
	ema17 = ema17array[len(ema17array)-1]
	ema18 = ema18array[len(ema18array)-1]
	ema19 = ema19array[len(ema19array)-1]
	ema20 = ema20array[len(ema20array)-1]
	ema21 = ema21array[len(ema21array)-1]
	ema22 = ema22array[len(ema22array)-1]
    
    #EMA 200
	ema23 = ema23array[len(ema23array)-1]
    
    #Fast EMA Color Rules
	if ema1 > ema2 and ema2 > ema3 and ema3 > ema4 and ema4 > ema5 and ema5 > ema6 and ema6 > ema7:
		colfastL = True
    
	if ema1 < ema2 and ema2 < ema3 and ema3 < ema4 and ema4 < ema5 and ema5 < ema6 and ema6 < ema7:
		colfastS = True
    
    #Slow EMA Color Rules
	if ema8 > ema9 and ema9 > ema10 and ema10 > ema11 and ema11 > ema12 and ema12 > ema13 and ema13 > ema14 and ema14 > ema15 and ema15 > ema16 and ema16 > ema17 and ema17 > ema18 and ema18 > ema19 and ema19 > ema20 and ema20 > ema21 and ema21 > ema22:
		colslowL = True
    
	if ema8 < ema9 and ema9 < ema10 and ema10 < ema11 and ema11 < ema12 and ema12 < ema13 and ema13 < ema14 and ema14 < ema15 and ema15 < ema16 and ema16 < ema17 and ema17 < ema18 and ema18 < ema19 and ema19 < ema20 and ema20 < ema21 and ema21 < ema22:
		colslowS = True
    
    #Fast EMA Final Color Rules
	if colfastL and colslowL:
		colFinal = "aqua"
	elif colfastS and colslowS:
		colFinal = "orange"
	else:
		colFinal = "gray"
        
    #Slow EMA Final Color Rules
	if colslowL:
		colFinal2 = "lime"
	elif colslowS:
		colFinal2 = "red"
	else:
		colFinal2 = "gray"
        
    #TRADING LOGIC
    #Buy If colFinal is aqua and colFinal2 is lime
	if colFinal == "aqua" and colFinal2 == "lime":
		print("	  BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		if countOpenTrades("B") == 0:
			print("	  Opening Buy Trade...")
			enter("B")
        
    #Sell If colFinal is orange and colFinal2 is red
	if colFinal == "orange" and colFinal2 == "red":
		print("	  SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
		if countOpenTrades("S") == 0:
			print("	  Opening Sell Trade...")
			enter("S")
        
    #EXIT STRTEGY
	if colFinal == "gray" and colFinal2 == "gray" :
		print("	  EXIT SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		print("	  EXIT BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
            
    
        
	
	# Print Price/Indicators
	print("Close Price: " + str(pricedata['bidclose'][len(pricedata)-1]))
	print("ColFinal: " + colFinal)
	print("ColFinal2: " + colFinal2)
	
		
	print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
	direction = True;
	if BuySell == "S":
		direction = False;
	try:
		opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount, time_in_force='GTC',order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop, trailing_step=trailstop_value)
	except:
		print("	  Error Opening Trade.")
	else:
		print("	  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				print("	  Closing tradeID: " + position['tradeId'])
				try:
					closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
				except:
					print("	  Error Closing Trade.")
				else:
					print("	  Trade Closed Successfully.")

# Returns number of Open Positions for symbol in the direction BuySell, returns total number of both Buy and Sell positions if no direction is specified
def countOpenTrades(BuySell=None):		
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	counter = 0
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				counter+=1
	return counter

Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy